# 2.7.3 The Glucose Minimal Model (Fitting Parameters)

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')
download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/chap06.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
from ModSimPy_Functions.chap06 import *
import pandas as pd
import numpy as np

In the class, we investigated the effect that the parameters ($k_1, k_2, etc...$ had our model, and discovered it was significant.  But how do we actually choose the parameters so that our model matches the data?

<br>

In this notebook, we'll use a well-known method to do just that.

## The Least Squares Method

Earlier in the course, we fit linear and exponential curves to our world population data by the guess-and-check method: guess at a slope, say, and then check to see how close it matches the data.  We could do the same here.  But with 4 parameters that gets very difficult, and we would never know if we've found the best set of parameters.  

<br>

So we'll use an algorithm to help us out: a SciPy function called `leastsq`, which stands for "least squares".  A least squares algorithm finds parameters by minimizing the sum of squared differences between the results of the model and the known data points.  The "squaring" serves two purposes: 

<br>

* it eliminates the possibility that a negative error might cancel out a positive error
* it heavily penalizes a data point that has a large error.

<br>

The sum of the squared errors becomes the *metric* we use to evaluate a set of parameters.  We can then minimize this metric to find the best parameters.

### Computing errors

Before we can minimize the error, though, we need a way to compute it.  To do this, we'll evaluate our model at each time step where we have known data, and then find the difference between the model and the known data at that time step.  In the end, we'll have a Series that contains the error at of these time steps.

<br>

To begin, we'll import our data and create a system, using the parameters and the `make_system` function from the previous notebook:

In [ ]:
filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/glucose_insulin.csv'
# If you are using this notebook offline, you will need to upload this data
# from the Images_and_Data folder.  Comment out the line above, and uncomment the
# line below this one, and run this cell
# filename = '../Images_and_Data/Data/glucose_insulin.csv'
data = pd.read_csv(filename, index_col='time')

Because of the way `leastsq()` is set up, we'll need to rework `make_system` so that it imports the parameters as a "tuple": a foundational Python object that is simply a list of objects.  Notice we also no longer need a `state` object, since `run_solve_ivp` takes care of that:

In [ ]:
def make_system(params, data):
    G0, k1, k2, k3, dt = params
    t_0 = data.index[0]
    t_end = data.index[-1]
    
    Gb = data.glucose[t_0]
    Ib = data.insulin[t_0]
    
    I = interp1d(data.insulin.index,data.insulin.values)

    init = pd.Series(dict(G=G0, X=0),dtype=np.float64)
    system = dict(init=init,
                  k1=k1,k2=k2,
                  k3=k3,dt=dt,
                  Gb=Gb, Ib=Ib, I=I,
                  t_0=t_0, t_end=t_end)
    
    return system

G0 = 270.0
k1 = 0.02
k2 = 0.02
k3 = 1.5e-05
dt = 2
params = G0, k1, k2, k3, dt
system = make_system(params,data)


Now we'll run the initial value problem solver.

In [ ]:
results, details = run_solve_ivp(system, slope_func, 
                                 t_eval=data.index)
details.message

Because we specify `t_eval=data.index`, the results are evaluated at the same time stamps as the data.

In [ ]:
results.tail()

We can plot the results like this.

In [ ]:
data.glucose.plot(style='o', alpha=0.5, label='data',
                  legend=True)
results.G.plot(style='-', color='C0', label='model',
               xlabel='Time (min)',
               ylabel='Concentration (mg/dL)',
              legend=True);

We can compute the errors by subtracting one data set from the other (this is why it was important that our results be evaluated at the same time steps as our data: otherwise, we couldn't add or subtract them):

In [ ]:
errors = results.G - data.glucose
pd.DataFrame(errors)

Most of the errors are under $10 mg/dL$.  But notice that during the first three time steps, the model does not fit the data. This is not surprising.  If you remember, the concentration of glucose in the blood stream initially varies widely (very high near the injection point, for inistance).  That's hard to capture it one number.  But as the glucose disperses in the blood stream, our model matches pretty well.

✅ Active reading: Explain in your own words why the first three data points in the model are not very accurate.  Considering that the errors are squared in the least squares method, what would these data points do to our "sum" of squared errors?

✅ ✅ Answer here.

### Defining an Error Function

Remember that when we used `root_scalar`, we defined an error function that calculated how close the final temperature of our model coffee was to our known experimental value of 70 degrees.

<br>

We are going to do the same thing here, but now we want the error func to return an array of errors (i.e. that does exactly what we did in the previous section).  `leastsq` will then run our model while changing the paramters, trying to make the sum of this array equal to zero.

<br>

Here's the error function that does this:

In [ ]:
def error_func(params,data):
    """Computes an array of errors to be minimized.
    
    params: 
    data: 
    returns: 
    """

    print(params)
    system = make_system(params, data)
    
    results, details = run_solve_ivp(system, slope_func, 
                                     t_eval=data.index)
    
    errors = results.G - data.glucose
    return errors.iloc[3:]

`error_func` uses the given parameters to make a `System` object, runs the simulation, and returns the errors.

<br>

But notice that it does not return all of the errors; rather, it uses the "index location" tool `iloc` to select only the elements with index 3 or more. In other words, it omits the elements with index 0, 1, and 2: the first three data points with large errors.  Since we don't expect the model to fit the data in this region, and since the errors in that region would overwhelm the errors elsewhere, we'll leave them out.

<br>

We can call `error_func` like this:

In [ ]:
errors = error_func(params, data)
errors.head()

✅ ✅ Active reading: Add inline comments (#) to `error_function`, and explain what params, data, and return are in the docstring at the top of the function.

### Running `leastsq`

Now we're ready to call `leastsq`.  As arguments, we pass `error_func`, the parameters where we want to start the search (this is why we had to change `make_system` to include `params`), and the data, which will be passed as an argument to `error_func`.

In [ ]:
best_params, fit_details = leastsq(error_func, params, data)

Each time `error_func` is called, it prints the parameters, so we can get a sense of how `leastsq` works.   Look at one of the printed columns: do you see how it starts at the given value (say, 270 for G0) and then starts to change?   Each line represents another "guess" by the `leastsq` algorithm, and with each guess, it searches out a smaller sum of the squares of the errors.  This process is called *iteration*.

<br>

Notice that in the last few iterations, the values of the parameters are changing by miniscule amounts: at this point, the algorithm is narrowing down the values to its final answer, which is the set of parameters that minimize the sum of the squares of the errors.

<br>

`leastsq` has two return values.  The first is an array with the best parameters:

In [ ]:
best_params

The second is an object with information about the results, including a success flag and a message.

In [ ]:
fit_details.success

In [ ]:
fit_details.mesg

This means that the last two iterations produced nearly identical results, and is a sign that the iteration process is complete.

### Checking our Results

Now that we have `best_params`, we can use it to make a `System` object, run the simulation, and see how accurate our results are.

In [ ]:
system2 = make_system(best_params, data)
results2, details = run_solve_ivp(system2, slope_func, t_eval=data.index)
print(details.message)
data.glucose.plot(style='o', alpha=0.5, label='data',
                 legend=True)
results.G.plot(style='-', color='C0', label='model',
               xlabel='Time (min)',
               ylabel='Concentration (mg/dL)',
              legend=True);


Now we can compute the errors directly.  What kind of value do we expect?

In [ ]:
errors2 = results2.G - data.glucose
errors2.head()

Notice that the errors are not zero.  This is because no function of the form that we are using is capable of going through each data point.  The curve is a "best fit", but it is not a *perfect* fit.  

<br>

Let's compare the error resulting from our initial parameters to the error resulting from the `best_params`.   To do this, we want to sum the square of these errors, a process that usually happens *within* the function `leastsq`.   Here are the results:

In [ ]:
from numpy import sum

init_param_error = sum(errors.iloc[3:]**2)
best_param_error = sum(errors2.iloc[3:]**2)
print("The sum of squared errors for the initial params is", round(init_param_error,1))
print("The sum of squared errors for the best params is", round(best_param_error,1))

The absolute size of these numbers is not meaningful: if we had used more time steps, the magnitude of the numbers would be larger.   But we can see that the sum of the squares went down noticeably, and could calculate a percent change:

In [ ]:
rel_change_error = (init_param_error-best_param_error)/init_param_error
print("The size of the sum of squares of the error decreased by", 
      round(rel_change_error*100,1), "percent")

---

<br>

## Summary

In this notebook we tested out the least squares method, an algorithm that helped us to minimize the difference between our model and our known data (i.e. the error of the model).   

<br>

Having validated our parameters by comparing our results to known data, we would be more confident predicting the behavior of the system when one of the conditions of the physical system (like the initial condition) changed.

<br>

---

### Exercise 1

How sensitive are the results to the starting guess for the parameters?  

<br>

Let's see what difference the initial value of G0 makes on the final results.  Run the notebook 3 times, and each time take note of the plot of results, the optimized value for G0, and sum of squared errors produced with the optimized parameters (the variable called `best_param_errors'):

<br>

* First, run it with the values we have been using all along ($G0 = 270$).
* Next, change G0 just by a bit and run the notebook ($G0 = 300$ rather than $270$).   
* Finally, change G0 by a lot (say $G0 = 800$).  

<br>

What happens to the recommended G0 value and overall error (i.e. the sum of squared errors) in each case?

<br>

Hint: you test a new set of parameters simply by changing the parameters at the top of the notebook, and running all the cells again.

✅ ✅  Describe your results (with numbers!) and try to explain why this might happen.  What does this suggest about algorithms like `leastsq`?  Can you always trust them?
